In [2]:
# # Pull Data

# # Snowflake , Get Data

# import snowflake.connector
# from snowflake.sqlalchemy import URL
# from sqlalchemy import create_engine
# # Sql Engine, where you input connection paramaters, User Name, password etc. Stuff you see in the snowflake gui top right. 

# # snowflake function
# def get_data_from_snowflake(sql):

#     engine = create_engine(URL(
#         account = 'cdm',
#         user = 'kchun',
#         password = 'Monarch219!',
#         database = 'EDWDBPRD',
#         #schema = 'public',
#         warehouse = 'ANALYTICS_PROD_WH',
#         role='DATA_SCIENCE_PROD_ADMIN_ROLE',
#         authenticator='externalbrowser'
#     ))

#     print('[INFO] connecting to Snowflake')
#     connection = engine.connect()
#     print('[INFO] connected, executing query...')

#     try:
#         df = pd.read_sql_query(sql, engine)
#         df.columns = df.columns.str.upper()
#         df.rename(index=str, columns={'INQUIRY': 'Inquiry','PLACEMENT':'placement'}, inplace=True)
#         print('[INFO] query executed')
#     finally:
#         connection.close()
#         engine.dispose()

#     return(df)

In [3]:
import pandas as pd
import numpy as np

In [4]:
# record prob output drawn to decide the argmax into a dataframe
prob_df = pd.DataFrame(columns=['p0','p1','p2','p3', \
                                'p4']);print(prob_df);print(0)

Empty DataFrame
Columns: [p0, p1, p2, p3, p4]
Index: []
0


In [5]:
prob_df

,p0,p1,p2,p3,p4


In [6]:
 #this is all duplicated below
print("\n Bayesian Bandit using Thompson Sampling \n")
print("Goal is to maximize the attempts on the highest converting variant")


N = 5  #Number of Variants


means = np.array([0.7, 0.3, 0.4, 0.5, .55])  # Define the conversion rates, unknown in real life

probs = np.zeros(N)  # random sample from posterior probability distribution for each machine (EV would be mean?)
            # this simply determines which one to pull(the handle of machine with the higest one of these will be pulled)
S = np.zeros(N, dtype=int)  # number successes each machine
F = np.zeros(N, dtype=int)  # number failures each machine
rnd = np.random.RandomState(221)  # for machine payouts and Beta


 Bayesian Bandit using Thompson Sampling 

Goal is to maximize the attempts on the highest converting variant


In [7]:
probs

array([0., 0., 0., 0., 0.])

In [8]:
S

array([0, 0, 0, 0, 0])

In [9]:
F

array([0, 0, 0, 0, 0])

In [10]:
rnd.beta(1,1) #will always be 0.14278824800173925 and then 0.873104102698192...

0.14278824800173925

In [11]:
# beta distribution conjugate prior Thompson sampling

In [12]:
np.mean(rnd.beta(1,1,30000000)) #usually will have 3 zeros to right of "5" OR 3 nine's to the right of "4"
#(due to pulling 30 MILLION numbers from beta distrib.)

0.5000090883712024

In [56]:
np.mean(rnd.beta(1,1,3000)) #usually has 1 zero to right of "5" or 1 nine to the right of "4"
#(due to only pulling 3k numbers from beta distrib.)

0.5004305418486744

In [57]:
np.mean(rnd.beta(5,5,3000)) #usually has 1 zero to right of "5" or 1 nine to the right of "4"
#(due to only pulling 3k numbers from beta distrib.)

0.5006319157561

In [58]:
np.mean(rnd.beta(66,66,3000)) #usually has 1 zero to right of "5" or 1 nine to the right of "4"
#(due to only pulling 3k numbers from beta distrib.)

0.4996357309077895

***
Note: no matter how many trials and samples (30000000 above) we use, the mean estimates of the lower converting handles are never very accurate -- that is b/c they are pulled so little - which is a good thing!

Next step is to set a random number of days that the win comes in at (loss always comes in after 100 days)

I can loop through a dict of all previous pulls (containing how long that win or loss will take to come in) and decrement it for each 1000 trials (assuming we send 1000 inquiries a day) -- when it hits zero, I pull it into the posterior beta(S,F) calc

show how even if distribution is not same bandit still works while A/B can NOT

splash A/B funnel C/D 4 combos
***

In [85]:
# record prob output drawn to decide the argmax into a dataframe

print("\n Bayesian Bandit using Thompson Sampling \n")
print("Goal is to maximize the attempts on the highest converting variant")


N = 5  #Number of Variants

trials = 10000   #how many total pulls do we want to do?

trial_data =[]  # array of trial #'s (for display at end)

machine_data=[] # array of which handles we pulled on each trial (for display at end)

win_loss=[] #array of whether we won or lost on each trial (for display at end)



means = np.array([0.9, 0.88, 0.85, 0.75, 0.5])  # Define the actual conversion rates for each machine (unknown in real life)


probs = np.zeros(N) #temp storage for single trial: random sample from posterior probability distribution for each machine(EV = the mean?)
   #NOTE: this simply determines which machine to pull(the handle of machine with the higest one of these will be pulled)

prob_df = pd.DataFrame(columns=['p0','p1','p2','p3','p4']) #holds the history of probs (above) from each trial



beta_mean = np.zeros(N) #temporary storage for a single trial: I will take the mean of a random sample from the same posterior distributions used above for probs (just to show it)

beta_mean_df = pd.DataFrame(columns=['p0','p1','p2','p3','p4']) #holds the history of beta_mean's (above) from each trial


S = np.zeros(N, dtype=int)  # running count of successes for each machine
F = np.zeros(N, dtype=int)  # running count of failures for each machine
pulls = np.zeros(N, dtype=int) # running count of pulls for each machine (just used at end for display)

rnd = np.random.RandomState()  #random series for machine payouts and Beta (put in a number to replicate series)

show_each_trial = 0#1 #change to 1 if you want to print each trial (otherwise only prints last one)

def prints(*args, **kwargs):
    if show_each_trial == 1:
        if kwargs=="":print(*args, **kwargs, end="")
        else: print(*args, **kwargs)
    elif trial >= trials-1:
        if kwargs=="":print(*args, *kwargs, end="")
        else: print(*args, **kwargs)

def samples():
  if show_each_trial == 1:
    return 30000#000
  elif trial >= trials-1:
    return 3000000
  else:
    return 3 #no need for precision if we aren't going to display the mean for this trial
 
for trial in range(trials):
    
    prints("\nTrial " + str(trial))
    
    trial_data.append(trial) #append this trial # to the trial_data array we defined above
    
    #for the new trial, clear out the probabilities we use to decide which machine to pull (see above desc for same variable)
    #probs = np.zeros(N)
    
    #pull from the posterior probability distribution for each machine to get the prob for it (again, just used to pick handle)
    for i in range(N): 
      
      probs[i] = rnd.beta(S[i] + 1, F[i] + 1) #since we are using a beta(1,1) prior, this is the posteror prob.
    
      beta_mean[i] = np.mean(rnd.beta(S[i] + 1, F[i] + 1, samples())) #last number is how many samples to pull to calc mean

      #global prob_df  #i guess this isn't needed anymore b/c I'm not modifying it inside a function
      if i == 4: #last machine
        columns = ['p0','p1','p2','p3','p4']  
        
        df = pd.DataFrame(probs, index=columns).T 
        prob_df = pd.concat([prob_df, df], ignore_index = True) #save the probs for this trial
        
        df = pd.DataFrame(beta_mean, index=columns).T
        beta_mean_df = pd.concat([beta_mean_df, df], ignore_index = True) #save the means of the dist's used to get the probs
        
      else:
        pass
  
    prints("means of posterior probability dist (this should get closer to actual over time)")
    for i in range(N):
        prints('%0.4f ' % beta_mean[i], end='')
        
    prints("")
    
    prints("sampling probs =  ")
    for i in range(N):
      prints("%0.4f  " % probs[i], end="")

      
    prints("")

    machine = np.argmax(probs)
    prints("Playing machine " + str(machine))
    
    machine_data.append(machine)
    # Updates success and fails in real life we can can update conversions , clicks, or whatever success of chosen variant
    
    p = rnd.random_sample()  # [0.0, 1.0)
    prints(p)
    if p < means[machine]:
          prints(" -- win")
          S[machine] += 1
          win_loss.append('W')
    else:
          prints(" -- lose")
          F[machine] += 1
          win_loss.append('L')

    prints("\nSuccess vector: ", end="")
    prints(S)
    prints("Failure vector: ", end="")
    prints(F)
    prints("Total pulls: ", end="")
    pulls=[S[i]+F[i] for i in range(len(S))]
    prints(pulls)
    



 Bayesian Bandit using Thompson Sampling 

Goal is to maximize the attempts on the highest converting variant

Trial 9999
means of posterior probability dist (this should get closer to actual over time)
0.8984 0.8891 0.8470 0.7200 0.4167 
sampling probs =  
0.9053  0.8977  0.8645  0.8408  0.6443  
Playing machine 0
0.7655076650672106
 -- win

Success vector: [7249 1442  215   35    4]
Failure vector: [819 179  38  13   6]
Total pulls: [8068, 1621, 253, 48, 10]


In [86]:
#put all the data together

trial_df=pd.DataFrame(trial_data,columns=['trials']).reset_index(drop=True)

machine_df=pd.DataFrame(machine_data,columns=['machine']).reset_index(drop=True)

win_loss_df=pd.DataFrame(win_loss,columns=['win/loss']).reset_index(drop=True)

final = pd.concat([trial_df, machine_df, win_loss_df, prob_df], axis=1, sort=False)

In [87]:
pd.set_option('display.max_rows', 30000)

In [88]:
final

,trials,machine,win/loss,p0,p1,p2,p3,p4
0,0,2,W,0.522084,0.104034,0.607576,0.439679,0.255089
1,1,2,W,0.068128,0.226921,0.497290,0.473170,0.374690
2,2,1,W,0.446302,0.877724,0.437262,0.721853,0.358934
3,3,1,W,0.733333,0.988775,0.898399,0.894890,0.872499
4,4,3,L,0.683769,0.696181,0.519708,0.865415,0.560992
5,5,0,L,0.923724,0.471884,0.668591,0.244016,0.293808
6,6,2,W,0.725649,0.955386,0.972420,0.285196,0.900491
7,7,2,W,0.048994,0.926640,0.998658,0.651320,0.008591
8,8,4,W,0.143775,0.487745,0.769810,0.495237,0.885035
9,9,2,W,0.693459,0.673072,0.977772,0.494479,0.900925


In [ ]:
def main():
    print(0)

if __name__ == "__main__":
  main()
    

In [ ]:
final